In [7]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
dataset_path = '/content/drive/My Drive/Music Genre Identification/genres/'

In [11]:
genres = ['metal', 'pop', 'reggae', 'rock','blues', 'classical', 'country', 'disco', 'hiphop', 'jazz' ]

In [12]:

def extract_features(dataset_path):
    audio, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
    mel = librosa.feature.melspectrogram(y=audio, sr=sr)
    contrast = librosa.feature.spectral_contrast(y=audio, sr=sr)
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(audio), sr=sr)

    return np.hstack([np.mean(mfcc.T, axis=0), np.mean(chroma.T, axis=0), np.mean(mel.T, axis=0),
                      np.mean(contrast.T, axis=0), np.mean(tonnetz.T, axis=0)])


In [13]:
X = []
y = []

for genre in genres:
    genre_path = os.path.join(dataset_path, genre)
    for filename in os.listdir(genre_path):
        if filename.endswith('.au'):
            file_path = os.path.join(genre_path, filename)
            y.append(genre)
            features = extract_features(dataset_path)
            X.append(features)

In [15]:
X = np.array(X)
y = np.array(y)

In [16]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
def create_model():
    model = keras.Sequential([
        keras.layers.Dense(1024, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.4),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.4),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dense(len(genres), activation='softmax')
    ])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [19]:
n_models = 3
models = []

for i in range(n_models):
    model = create_model()
    model.fit(X_train, y_train, epochs=450, batch_size=16, validation_data=(X_test, y_test), verbose=0)
    models.append(model)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
from sklearn.metrics import accuracy_score
def ensemble_predictions(models, X_test):
    predictions = [model.predict(X_test) for model in models]
    avg_predictions = np.mean(predictions, axis=0)
    return np.argmax(avg_predictions, axis=1)


y_pred = ensemble_predictions(models, X_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [21]:
from sklearn.metrics import accuracy_score
ensemble_accuracy = accuracy_score(y_test, y_pred)
print(f'Ensemble accuracy: {ensemble_accuracy}')

Ensemble accuracy: 0.675


In [22]:
# Save the trained model to Google Drive for future use
model_save_path = '/content/drive/My Drive/Music Genre Identification/Music_Genre_Identification_model.keras'
model.save(model_save_path)